In [ ]:
from model import layers, activations, networks, utils, optimizers, losses, paddings, metrics
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
import os

In [ ]:
def loadImages(n):
    filepath = 'datasets\\catsanddogs\\Petimages'
    list_x = []
    list_y = []

    c = 0
    for category in ['Cat', 'Dog']:
        path = filepath + '\\' + category
        for i, image in enumerate(os.listdir(path)):
            if i >= n / 2: break
            if image[-3:] != 'jpg': continue
            image_rgb = Image.open(path + '\\' + image)
            image_resized = image_rgb.resize((128, 128))
            image_arr = np.asarray(image_resized)
            if len(image_arr.shape) != 3: continue
            list_x.append(image_arr / 255)
            list_y.append(0 if category == 'Cat' else 1)
            c = c + 1
            print(f'{c}/{n} images loaded', end='\r')
    return np.array(list_x), np.array(list_y)

X, Y = loadImages(500)

In [ ]:
Y = np.expand_dims(Y, -1)
print(X.shape)
print(Y.shape)

In [ ]:
X, Y = utils.shuffle(X, Y)
l = int(len(X) * 0.7)
[x_train, x_test] = np.split(X, [l])
[y_train, y_test] = np.split(Y, [l])

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
model = networks.FeedForward([
    layers.Input(input_shape=(128, 128, 3)),
    layers.Convolution(nr_kernels=8, kernel_size=(3, 3), activation=activations.ReLu),
    layers.MaxPooling(pooling_window=(2, 2)),
    layers.Convolution(nr_kernels=16, kernel_size=(3, 3), activation=activations.ReLu),
    layers.MaxPooling(pooling_window=(2, 2)),
    layers.Flatten(),
    layers.Dense(nr_neurons=128, activation=activations.ReLu),
    layers.Dropout(0.1),
    layers.Dense(nr_neurons=1, activation=activations.Sigmoid),
    layers.Output()
    ])

In [ ]:
model.compile(
    optimizer=optimizers.stochastic_gradient_descent(learning_rate=0.01, momentum=0.9, nesterov=True),
    loss=losses.root_mean_square,
    metric=metrics.Accuracy
    )

In [ ]:
model.summary()

In [ ]:
model.train(x_train, y_train, epochs=50, batch_size=50)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
image = X[2]
plt.imshow(image)
model.predict(image)

In [ ]:
layer = 1
for i in range(model.layers[layer].k):
    plt.imshow(model.layers[layer].o[:, :, i], cmap='gray')
    plt.show()